# Freesurfer space to native space using `mri_vol2vol`

BMED360-2021: `freesurfer-to-native-space.ipynb`

In [1]:
%matplotlib inline
import os
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from os.path import expanduser, join, basename, split
import sys
sys.path.append('.')   # path to utils.py
import utils
import time
import shutil  # copy files
cwd = os.getcwd()

### We will use the `fs711_subjects` Freesurfer tree previously run for the `bids_bg_bmed360` sample

In [2]:
fs711_home = '/usr/local/freesurfer'
working_dir = join(cwd, 'data')
bids_dir = '%s/bids_bg_bmed360' % (working_dir)
fs711_subj = '%s/fs711_subjects' % (working_dir)
dmri_res = '%s/dmri_results' % (working_dir)

In [3]:
if not os.path.exists(dmri_res):
    os.makedirs(dmri_res)
else:
    print('subdirectory dmri_results already exists')

subdirectory dmri_results already exists


The Freesurfer environment:
```
%%bash -s  '/usr/local/freesurfer' './data/fs711_subjects'

echo $1
echo $2

FREESURFER_HOME=${1}; export FREESURFER_HOME
PATH=${FREESURFER_HOME}/bin:${PATH}; export PATH
SUBJECTS_DIR=${2}; export SUBJECTS_DIR
FSLDIR=/usr/local/fsl; export FSLDIR
PATH=${FSLDIR}/bin:${PATH}; export PATH
. ${FSLDIR}/etc/fslconf/fsl.sh
source ${FREESURFER_HOME}/SetUpFreeSurfer.sh
```

## How to Convert from FreeSurfer Space Back to Native Anatomical Space
See: https://surfer.nmr.mgh.harvard.edu/fswiki/FsAnat-to-NativeAnat

Question: I have successfully run a subject's data through [FreeSurfer](https://surfer.nmr.mgh.harvard.edu/fswiki/FreeSurfer). 
FreeSurfer creates volumes in 1 mm$^3$, 256$^3$ space, but I want the FreeSurfer results in the space of my original anatomical. How do I do this?<br>

The exact command you use depends on what you want to convert, an image (like brain.mgz) or a segmentation (like aseg.mgz).

For an image:

In [4]:
# cd $SUBJECTS_DIR/<subjid>/mri
# mri_vol2vol --mov brain.mgz --targ rawavg.mgz --regheader --o brain-in-rawavg.mgz --no-save-reg

For a segmentation (aseg.mgz, aparc+aseg.mgz, wmparc.mgz, etc):

In [5]:
# cd $SUBJECTS_DIR/<subjid>/mri
# mri_label2vol --seg aseg.mgz --temp rawavg.mgz --o aseg-in-rawavg.mgz --regheader aseg.mgz

Map the surface to the native space:

In [6]:
# mri_surf2surf --sval-xyz pial --reg register.native.dat rawavg.mgz --tval lh.pial.native --tval-xyz rawavg.mgz --hemi lh --s subjectname

The output will be stored in $SUBJECTS_DIR/subjectname/surf/lh.pial.native and can be viewed with freeview rawavg.mgz -f ../surf/lh.pial.native<br>
To verify that this worked, run

In [7]:
# freeview -v rawavg.mgz -f lh.pial.native

In [8]:
MRI_VOL2VOL = '%s/bin/mri_vol2vol' % (fs711_home)
print(os.popen(MRI_VOL2VOL).read())


mri_vol2vol

  --mov  movvol       : input (or output template with --inv)
  --targ targvol      : output template (or input with --inv)
  --o    outvol       : output volume
  --disp dispvol      : displacement volume
  --downsample N1 N2 N3 : downsample input volume (do not include a targ or regsitration)
         sets --fill-average, --fill-upsample 2, and --regheader

  --reg  register.dat : tkRAS-to-tkRAS matrix   (tkregister2 format)
  --lta  register.lta : Linear Transform Array (usually only 1 transform)
  --lta-inv  register.lta : LTA, invert (may not be the same as --lta --inv with --fstal)
  --fsl  register.fsl : fslRAS-to-fslRAS matrix (FSL format)
  --xfm  register.xfm : ScannerRAS-to-ScannerRAS matrix (MNI format)
  --regheader         : ScannerRAS-to-ScannerRAS matrix = identity
  --mni152reg         : target MNI152 space (need FSL installed)
  --s subject         : set matrix = identity and use subject for any templates

  --inv               : sample from targ to mov


In [9]:
def my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype):
    """
    Ex. 
    cd $SUBJECTS_DIR/<subjid>/mri
    mri_vol2vol --mov brain.mgz --targ rawavg.mgz --regheader --o brain-in-rawavg.mgz --no-save-reg
    --interp interptype : interpolation cubic, trilin, nearest (def is trilin)
    """
    
    fs_mri = join('%s' % (subj_dir), 'sub_%d_tp%d/mri' % (sub, ses))  
    cmd = [
            MRI_VOL2VOL,
            '--mov', '%s/%s.mgz' % (fs_mri, inp_image),
            '--targ', '%s' % (targ_image),
            '--regheader', 
            '--interp', '%s' % (interptype),
            '--o', '%s/sub_%d_tp%d_%s_in_%s.nii.gz' % (out_dir, sub, ses, inp_image, targ_name),
            '--no-save-reg']
          #  ' 2>', error_output_log,'>', output_log]   
    cmd_str = " ".join(cmd)
    #print('cmd_str = \n%s\n' % cmd_str)
        
    # EXECUTE
    os.system(cmd_str)

### Testing the native space conversion on one subject (sub_102_tp1) 
**using the `sub-102_ses-1_T1w.nii.gz` in the `bids_bg_bmed360` tree as target image**

In [10]:
subj_dir = fs711_subj
out_dir = dmri_res

sub = 102
ses = 1

targ_image = '%s/sub-%d/ses-%d/anat/sub-%d_ses-%d_T1w.nii.gz' % (bids_dir, sub, ses, sub, ses)
targ_name = 'native_space'

**Use the `my_mri_vol2vol()`function on different source images and masks using approriate interpolation ('trilinear' and 'nearest neighbour')**

In [11]:
%%time

shutil.copy2(targ_image,out_dir)  # copy the original anatomy file in bids tree to out_dir

interptype = 'trilin'
inp_image = 'orig'
my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

interptype = 'trilin'
inp_image = 'brain'
my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

interptype = 'nearest'
inp_image = 'ribbon'
my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

interptype = 'nearest'
inp_image = 'aseg'
my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

interptype = 'nearest'
inp_image = 'wmparc'
my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

interptype = 'nearest'
inp_image = 'aparc+aseg'
my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

CPU times: user 4.59 ms, sys: 8.94 ms, total: 13.5 ms
Wall time: 8.6 s


#### Run the native space conversion on all subjects and sessions using the `_T1_biascorr_brain.nii.gz` image obtained from `03-fsl-anat.ipynb` as target image.

In [13]:
%%time
subj_dir = fs711_subj
bids_dir = bids_dir
out_dir = dmri_res
targ_name = 'native_space'

for sub in [102, 103, 111, 123]:
    for ses in [1, 2]:
        
        print(f'Computing sub:{sub} ses:{ses}')
            
        targ_image = join(bids_dir,'sub-%d/ses-%d/anat/sub-%d_ses-%d_T1w.nii.gz' % (sub, ses, sub, ses))
        shutil.copy2(targ_image,out_dir)  # copy the original anatomy file in bids tree to out_dir

        inp_image = 'orig'
        interptype = 'trilin'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)
        
        inp_image = 'brain'
        interptype = 'trilin'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

        inp_image = 'brainmask'
        interptype = 'nearest'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)
                       
        inp_image = 'ribbon'
        interptype = 'nearest'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

        inp_image = 'aseg'
        interptype = 'nearest'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

        inp_image = 'wmparc'
        interptype = 'nearest'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

        inp_image = 'aparc+aseg'
        interptype = 'nearest'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

Computing sub:102 ses:1
Computing sub:102 ses:2
Computing sub:103 ses:1
Computing sub:103 ses:2
Computing sub:111 ses:1
Computing sub:111 ses:2
Computing sub:123 ses:1
Computing sub:123 ses:2
CPU times: user 10 ms, sys: 51.9 ms, total: 61.9 ms
Wall time: 1min 15s
